In [1]:
import numpy as np

merged_bar = np.load("data/merged_bar.npy")
merged_bar.shape

(15063, 6)

In [2]:
import pandas as pd

from custom_indicators.all_features import feature_bundle

features = feature_bundle(merged_bar, sequential=True)
df_features = pd.DataFrame(features, index=merged_bar[:, 0].astype(int))
print(df_features.shape)
print(f"max nan: {df_features.isna().sum().max()}")
df_features.head()

(15063, 3631)
max nan: 1844


,adx_7,adx_7_dt,adx_7_ddt,adx_7_lag1,adx_7_lag2,adx_7_lag3,adx_7_lag4,adx_7_lag5,adx_7_lag6,adx_7_lag7,...,williams_r_ddt_lag10,williams_r_ddt_lag11,williams_r_ddt_lag12,williams_r_ddt_lag13,williams_r_ddt_lag14,williams_r_ddt_lag15,williams_r_ddt_lag16,williams_r_ddt_lag17,williams_r_ddt_lag18,williams_r_ddt_lag19
1577874960000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577896860000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577923140000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577944080000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1577967720000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = df_features.iloc[1844:]
print(df.shape)
df.isna().sum().max()

(13219, 3631)


0

In [5]:
df.to_parquet("data/features.parquet")

### 确认实际的nan数量

In [1]:
import numpy as np

from custom_indicators.prod_indicator.emd.cls_vmd_indicator import VMD_NRBO
from custom_indicators.prod_indicator.wavelets.cls_cwt_swt import CWT_SWT

merged_bar = np.load("data/merged_bar.npy")
print(merged_bar.shape)

vmd_nrbo = VMD_NRBO(merged_bar, 32, sequential=True)
cwt_swt = CWT_SWT(merged_bar, 32, sequential=True)

vmd_nrbo_single = VMD_NRBO(merged_bar, 32, sequential=False)
cwt_swt_single = CWT_SWT(merged_bar, 32, sequential=False)

(15063, 6)


In [7]:
single_res = cwt_swt_single.res(dt=True, lag=1)
sequential_res = cwt_swt.res(dt=True, lag=1)[-1]

single_res - sequential_res

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [3]:
vmd_nrbo_single.res()

array([[-660.2680612 ,  136.15266516,   67.72702177]])

In [ ]:
import numpy as np

from custom_indicators.prod_indicator._indicator_base._cls_ind import _fill_gap

dt_result = [i[1:] - i[:-1] for i in cwt_swt.raw_result]
dt_result = np.array([i[-1] for i in dt_result])
dt_result = _fill_gap(dt_result, cwt_swt.candles)

In [11]:
dt_result.shape

(15063, 21)

In [1]:
import json

with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "r") as f:
    feature_info = json.load(f)

all_features = []
all_features.extend(feature_info["meta"]["meta"])
all_features.extend(feature_info["side"]["long"])
all_features.extend(feature_info["side"]["short"])

all_features = sorted(list(set(all_features)))
len(all_features)

343

In [2]:
import pandas as pd

df = pd.read_parquet("data/features.parquet")
df[all_features].isna().sum().sort_values(ascending=False)

ac_1                                0
price_variance_ratio_dt_lag3        0
reactivity_lag3                     0
reactivity_lag2                     0
reactivity_lag1                     0
                                   ..
evenbetter_sinewave_long            0
ehlers_early_onset_trend_lag15      0
ehlers_early_onset_trend_lag1       0
ehlers_early_onset_trend_dt_lag2    0
williams_r_lag6                     0
Length: 343, dtype: int64

### 新特征探索

In [6]:
import numpy as np

merged_bar = np.load("data/merged_bar.npy")
close_prices = merged_bar[:, 2]

In [9]:
from custom_indicators.prod_indicator.wavelets.cwt_swt import cwt

res = cwt(merged_bar, window=32, source_type="close", sequential=False)
np.isnan(res).sum()

0

In [17]:
res.ndim

1